In [ ]:
import os
import tensorflow as tf

os.environ['XLA_FLAGS'] = '--xla_gpu_strict_conv_algorithm_picker=false'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # 추가: GPU 메모리 제한 (선택사항)
        # tf.config.set_logical_device_configuration(
        #     gpus[0],
        #     [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
        
        print(f"GPU 설정 완료: {len(gpus)}개")
    except RuntimeError as e:
        print(e)
        
print(f"현재 정책: {tf.keras.mixed_precision.global_policy()}")
print(f"GPU 장치: {tf.config.list_physical_devices('GPU')}")

: 

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import gzip
import gc

import tensorflow as tf
from tensorflow.keras import datasets, utils, layers, models, optimizers
import tensorflow.keras.backend as K

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     for gpu in gpus:
#         tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
def load_fashion_mnist(root_dir):
    data_dir = root_dir
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"Fashion MNIST Not Found!: {data_dir}")

    train_images_path = os.path.join(data_dir, 'train-images-idx3-ubyte.gz')
    train_labels_path = os.path.join(data_dir, 'train-labels-idx1-ubyte.gz')
    test_images_path = os.path.join(data_dir, 't10k-images-idx3-ubyte.gz')
    test_labels_path = os.path.join(data_dir, 't10k-labels-idx1-ubyte.gz')

    with gzip.open(train_images_path, 'rb') as f:
        x_train = np.frombuffer(f.read(), dtype=np.uint8, offset=16)
        x_train = x_train.reshape(-1, 28, 28)

    with gzip.open(train_labels_path, 'rb') as f:
        y_train = np.frombuffer(f.read(), dtype=np.uint8, offset=8)

    with gzip.open(test_images_path, 'rb') as f:
        x_test = np.frombuffer(f.read(), dtype=np.uint8, offset=16)
        x_test = x_test.reshape(-1, 28, 28)

    with gzip.open(test_labels_path, 'rb') as f:
        y_test = np.frombuffer(f.read(), dtype=np.uint8, offset=8)

    return (x_train, y_train), (x_test, y_test)

def preprocess(imgs):
    imgs = imgs.astype("float32") / 255.0
    imgs = np.pad(imgs, ((0, 0), (2, 2), (2, 2)), constant_values=0.0)
    imgs = np.expand_dims(imgs, -1)
    return imgs

In [ ]:
IMAGE_SIZE = 32
CHANNELS = 1
BATCH_SIZE = 32
BUFFER_SIZE = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 2
EPOCHS = 3

In [ ]:
root_dir = '/mnt/d/datasets/fashion_mnist'
(x_train,y_train), (x_test,y_test) = load_fashion_mnist(root_dir)

x_train = preprocess(x_train)
x_test = preprocess(x_test)

print(f"train images: {type(x_train)} {x_train.shape}, dtype={x_train.dtype}, min={x_train.min()}, max={x_train.max()}")
print(f"train labels: {type(y_train)} {y_train.shape}, dtype={y_train.dtype}, min={y_train.min()}, max={y_train.max()}")

In [ ]:
# Encoder
encoder_input = layers.Input(shape=(32, 32, 1), name = "encoder_input")
x = layers.Conv2D(32, (3, 3), strides = 2, activation = 'relu', padding="same")(encoder_input)
x = layers.Conv2D(64, (3, 3), strides = 2, activation = 'relu', padding="same")(x)
x = layers.Conv2D(128, (3, 3), strides = 2, activation = 'relu', padding="same")(x)
shape_before_flattening = K.int_shape(x)[1:]
x = layers.Flatten()(x)
encoder_output = layers.Dense(2, name="encoder_output")(x)
encoder = models.Model(encoder_input, encoder_output)

In [ ]:
# Decoder
decoder_input = layers.Input(shape=(2,), name="decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(128, (3, 3), strides=2, activation = 'relu', padding="same")(x)
x = layers.Conv2DTranspose(64, (3, 3), strides=2, activation = 'relu', padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation = 'relu', padding="same")(x)
decoder_output = layers.Conv2D(1, (3, 3), strides = 1, activation="sigmoid",
    padding="same", name="decoder_output")(x)
decoder = models.Model(decoder_input, decoder_output)

In [ ]:
# Autoencoder
autoencoder = models.Model(encoder_input, decoder(encoder_output))
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")
autoencoder.fit(x_train, x_train, epochs=5, batch_size=100, shuffle=True, 
    validation_data=(x_test, x_test))

In [ ]:
# K.clear_session()
# gc.collect()